In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import sys
import os
import pandas as pd
import backtrader as bt
import matplotlib

b:\Projects\portfolio_management\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [2]:
# Download latest version
path = kagglehub.dataset_download("khalilvandian/portfolio-management")

# List all files and directories in the downloaded dataset path
files = os.listdir(path)
print("Files in dataset directory:", files)

Files in dataset directory: ['AVGO.csv', 'AXP.csv', 'BAC.csv', 'CB.csv', 'CMG.csv', 'EA.csv', 'EBAY.csv', 'GRMN.csv', 'IBM.csv', 'IT.csv', 'LEG.csv', 'MHK.csv', 'MS.csv', 'ORLY.csv', 'XL.csv']


In [3]:
def clean_stock_data(filepath):
    df = pd.read_csv(filepath)

    # Standardize column names (lowercase, no special characters)
    df.columns = [col.strip().lower().replace('/', '_').replace(' ', '_') for col in df.columns]

    # Rename 'close/last' to 'close' if needed
    if 'close_last' in df.columns:
        df.rename(columns={'close_last': 'close'}, inplace=True)

    # Remove dollar signs and convert to float
    df['close'] = df['close'].replace('[\\$,]', '', regex=True).astype(float)
    df['open'] = df['open'].replace('[\\$,]', '', regex=True).astype(float)
    df['high'] = df['high'].replace('[\\$,]', '', regex=True).astype(float)
    df['low'] = df['low'].replace('[\\$,]', '', regex=True).astype(float)

    # Parse dates
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)

    return df

In [4]:
stock_data_dict = {}
full_date_index = pd.date_range(start="2015-06-16", end="2025-06-13", freq='B')

for file in files: 
    ticker_name = file.replace(".csv", "").strip()
    file_path = os.path.join(path, file)

    stock_df = clean_stock_data(file_path)
    stock_df = stock_df.reindex(full_date_index)
    stock_df.ffill(inplace=True) 
    stock_df.fillna(0, inplace=True)

    stock_data_dict[ticker_name] = stock_df


# Add no risk option
# copy 1st df and change all values to a constant value
no_risk = stock_data_dict["AVGO"].copy()
no_risk["close"] = 1
no_risk["open"] = 1
no_risk["high"] = 1
no_risk["low"] = 1
no_risk["volume"] = 0
stock_data_dict["NoRisk"] = no_risk

tickers = list(stock_data_dict.keys())
print(tickers)

['AVGO', 'AXP', 'BAC', 'CB', 'CMG', 'EA', 'EBAY', 'GRMN', 'IBM', 'IT', 'LEG', 'MHK', 'MS', 'ORLY', 'XL', 'NoRisk']


In [5]:
no_risk

,close,volume,open,high,low
2015-06-16,1,0,1,1,1
2015-06-17,1,0,1,1,1
2015-06-18,1,0,1,1,1
2015-06-19,1,0,1,1,1
2015-06-22,1,0,1,1,1
...,...,...,...,...,...
2025-06-09,1,0,1,1,1
2025-06-10,1,0,1,1,1
2025-06-11,1,0,1,1,1
2025-06-12,1,0,1,1,1


In [6]:
# Find and print tickers with any NaN in their "close" column
for ticker, df in stock_data_dict.items():
	if df["close"].isna().any():
		print(f"{ticker} has NaN values in 'close':")
		print(df[df["close"].isna()])

# Constant Weight Portfolio 

In [7]:
# create dataframe for the investment
investment_strategy = pd.DataFrame(tickers, columns=["Symbol"])

# set budget as 1 dollar
budget = 100000

investment_strategy["Weight"] = round(1/len(investment_strategy), 3)
investment_strategy["Position_Size"] = budget * investment_strategy["Weight"]

investment_strategy

,Symbol,Weight,Position_Size
0,AVGO,0.062,6200.0
1,AXP,0.062,6200.0
2,BAC,0.062,6200.0
3,CB,0.062,6200.0
4,CMG,0.062,6200.0
5,EA,0.062,6200.0
6,EBAY,0.062,6200.0
7,GRMN,0.062,6200.0
8,IBM,0.062,6200.0
9,IT,0.062,6200.0


In [36]:
class BuyAndHoldCustom(bt.Strategy):

    # Add a parameters class to accept allocations
    params = (('allocations', {}),)

    def __init__(self):
        self.bought = {}
        # Access the allocations from the parameters
        self.allocations = self.p.allocations

    def next(self):
        for data in self.datas:
            name = data._name
            if not self.getposition(data).size and name not in self.bought:
                cash = self.broker.get_cash()
                allocation = self.params.allocations[name]
                size = int(allocation / data.close[0])
                self.buy(data=data, size=size)
                self.bought[name] = True

    def stop(self):
        # Called at the end of the backtest
        for data in self.datas:
            position = self.getposition(data)
            if position.size > 0:
                self.sell(data=data, size=position.size)
                print(f"SELL at END: {data._name} @ {data.close[0]:.2f}, Size: {position.size}")

def run_custom_backtest(dataframes_dict, allocations_dict, start_date, end_date, budget):
    cerebro = bt.Cerebro()
    cerebro.broker.set_cash(budget)

    filtered_dataframes = {}
    for stock_name, df in dataframes_dict.items():
        df = df.copy()
        # df['Date'] = pd.to_datetime(df['date'])
        # df.set_index('Date', inplace=True)
        df = df.loc[start_date:end_date]
        df.columns = df.columns.str.capitalize()  # Ensure 'Close' column exists
        filtered_dataframes[stock_name] = df
        feed = bt.feeds.PandasData(dataname=df)
        cerebro.adddata(feed, name=stock_name)

    # Pass allocations into strategy as parameter
    cerebro.addstrategy(BuyAndHoldCustom, allocations=allocations_dict)

    start_value = cerebro.broker.getvalue()
    cerebro.run()
    end_value = cerebro.broker.getvalue()
    end_cash = cerebro.broker.getcash()

    return {
        'start_value': start_value,
        'end_value': end_value,
        'total_gain': end_value - start_value,
        'percent_gain': (end_value - start_value) / (start_value-end_cash) * 100,
        'end_cash': end_cash,
        'cerebro': cerebro
    }


In [37]:
position_size_dict = {}
for index, row in investment_strategy.iterrows():
    position_size_dict[row['Symbol']] = row['Position_Size']

# create company dicts
dataframes_dict = stock_data_dict.copy()

result = run_custom_backtest(
    dataframes_dict=dataframes_dict,
    allocations_dict=position_size_dict,
    start_date='2024-12-01',
    end_date='2025-05-31',
    budget=budget
)

print("Initial Value:", result['start_value'])
print("Final Value:", result['end_value'])
print("Total Gain:", result['total_gain'])
print("Percent Gain:", result['percent_gain'], "%")

SELL at END: AVGO @ 242.07, Size: 37
SELL at END: AXP @ 294.05, Size: 20
SELL at END: BAC @ 44.13, Size: 131
SELL at END: CB @ 297.20, Size: 21
SELL at END: CMG @ 50.08, Size: 102
SELL at END: EA @ 143.78, Size: 37
SELL at END: EBAY @ 73.17, Size: 98
SELL at END: GRMN @ 202.97, Size: 29
SELL at END: IBM @ 259.06, Size: 27
SELL at END: IT @ 436.42, Size: 11
SELL at END: LEG @ 9.06, Size: 486
SELL at END: MHK @ 100.61, Size: 45
SELL at END: MS @ 128.03, Size: 47
SELL at END: ORLY @ 91.17, Size: 74
SELL at END: XL @ 0.10, Size: 41333
SELL at END: NoRisk @ 1.00, Size: 6200
Initial Value: 100000
Final Value: 96057.76670000001
Total Gain: -3942.2332999999926
Percent Gain: -4.021575235306036 %


In [38]:
# Generate plot
cerebro = result["cerebro"]
# cerebro.plot(numfigs=16)

try:
	figs = cerebro.plot()  # You can choose style: line, candlestick, ohlc

	# Set large size and save the first figure
	fig = figs[0][0]  # First subplot in first figure
	fig.set_size_inches(24, 60)  # Width, Height in inches
	fig.savefig("large_backtest_plot.png", dpi=100)  # Save with high resolution
except ValueError as e:
	print("Plotting failed due to:", e)
	print("Try removing tickers with NaN values from your data.")

<IPython.core.display.Javascript object>